# PSO based Parking Recommendation System

This notebook provides a parking recommendation system that selects an optimal parking lot near a given destination using a heuristic optimization approach.

**Key Features:**
- Calculates the distance of each parking lot from the destination using the Haversine formula.
- Scores parking lots based on weighted criteria (distance, cost, and accessibility).
- Applies a search radius to limit the number of considered parking lots.
- Attempts an extended search radius if none are found within the initial search range.
- Provides a folium-based map visualization of recommended parking lots, highlighting the best option.

**Requirements:**
- A CSV file named `Parking_Lot_Dataset.csv` containing:
  - `id`, `lat`, `lng`, `rate_hourly`

**Instructions:**
1. Update the `destination` variable with your desired coordinates.
2. Run all cells to generate recommendations and visualize results.
3. The results will be saved as `parking_recommendation.html` file.

---



## Imports and Setup

Below we import all the necessary libraries and define supporting functions and classes.

In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import folium
from typing import List, Tuple, Dict
import random

## Class Definition

`ParkingOptimizer` is responsible for:
- Filtering parking lots based on a given search radius.
- Computing the Haversine distance for each lot to the destination.
- Calculating a fitness score for each parking lot.
- Providing recommendations and visualizations.

In [2]:
class ParkingOptimizer:
    def __init__(self, parking_data: pd.DataFrame, destination: Tuple[float, float],
                 weights: Dict[str, float] = None, search_radius: int = 800):
        """
        Initialize the parking optimizer
        """
        self.parking_data = parking_data
        self.destination = destination
        self.weights = weights or {'distance': 0.5, 'cost': 0.3, 'accessibility': 0.2}
        self.search_radius = search_radius

        # Calculate distances for all parking lots
        self.parking_data['distance'] = self.parking_data.apply(
            lambda row: self.haversine_distance(
                row['lat'], row['lng'],
                self.destination[0], self.destination[1]
            ),
            axis=1
        )

        # Filter parking lots within search radius
        self.filtered_data = self.parking_data[self.parking_data['distance'] <= self.search_radius].copy()

        # PSO parameters
        self.num_particles = min(20, len(self.filtered_data))
        self.max_iterations = 50
        self.c1 = 2.0
        self.c2 = 2.0
        self.w = 0.7

    def haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """Calculate the distance between two points on Earth"""
        R = 6371000  # Earth's radius in meters
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        return R * c

    def calculate_fitness(self, parking_lot: pd.Series) -> float:
        """Calculate fitness score for a parking lot"""
        max_distance = self.filtered_data['distance'].max()
        max_cost = self.filtered_data['rate_hourly'].max()

        distance_score = (max_distance - parking_lot['distance']) / max_distance if max_distance > 0 else 1
        cost_score = (max_cost - parking_lot['rate_hourly']) / max_cost if max_cost > 0 else 1
        accessibility_score = random.uniform(0.7, 1.0)

        fitness = (
            self.weights['distance'] * distance_score +
            self.weights['cost'] * cost_score +
            self.weights['accessibility'] * accessibility_score
        )
        return fitness

    def get_recommendations(self) -> pd.DataFrame:
        """Get all parking lots with their fitness scores"""
        if len(self.filtered_data) == 0:
            return pd.DataFrame()

        self.filtered_data['fitness'] = self.filtered_data.apply(self.calculate_fitness, axis=1)
        return self.filtered_data.sort_values('fitness', ascending=False)

    def visualize_results(self, recommendations: pd.DataFrame) -> folium.Map:
        """Create a map visualization"""
        m = folium.Map(location=[self.destination[0], self.destination[1]], zoom_start=15)

        # Add destination marker
        folium.Marker(
            [self.destination[0], self.destination[1]],
            popup='Destination',
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

        # Add search radius circle
        folium.Circle(
            radius=self.search_radius,
            location=[self.destination[0], self.destination[1]],
            color='red',
            fill=True,
            fillOpacity=0.1
        ).add_to(m)

        # Add all parking lots (black markers)
        for _, lot in self.parking_data.iterrows():
            popup_text = f"""
                ID: {lot['id']}<br>
                Rate: ${lot['rate_hourly']}/hr<br>
                Distance: {lot['distance']:.0f}m
            """
            folium.Marker(
                [lot['lat'], lot['lng']],
                popup=popup_text,
                icon=folium.Icon(color='black')
            ).add_to(m)

        # Add recommended parking lots (green for best, blue for others)
        if not recommendations.empty:
            best_lot = recommendations.iloc[0]
            for _, lot in recommendations.iterrows():
                color = 'green' if lot['id'] == best_lot['id'] else 'blue'
                popup_text = f"""
                    ID: {lot['id']}<br>
                    Rate: ${lot['rate_hourly']}/hr<br>
                    Distance: {lot['distance']:.0f}m<br>
                    Fitness Score: {lot['fitness']:.3f}
                """
                folium.Marker(
                    [lot['lat'], lot['lng']],
                    popup=popup_text,
                    icon=folium.Icon(color=color)
                ).add_to(m)

        return m

## Main Function

`parking_recommendation`:
- Loads the parking data (here we keep the same structure to avoid altering logic).
- Instantiates `ParkingOptimizer`.
- Retrieves recommendations.
- Prints out the best lot and other available lots.
- Visualizes and saves results as `parking_recommendation.html`.


In [3]:
def parking_recommendation(destination: Tuple[float, float], search_radius: int = 800) -> None:
    """Main function to run parking optimization"""
    parking_data = pd.read_csv('Parking_Lot_Dataset.csv')

    weights = {
        'distance': 0.5,
        'cost': 0.3,
        'accessibility': 0.2
    }

    # Try with initial search radius
    optimizer = ParkingOptimizer(parking_data, destination, weights, search_radius)
    recommendations = optimizer.get_recommendations()

    if recommendations.empty:
        print(f"No parking lots are available within {search_radius} meters radius.")
        # Try with 1.5 times search radius
        extended_radius = int(search_radius * 1.5)
        print(f"Trying extended radius of {extended_radius} meters...")
        optimizer = ParkingOptimizer(parking_data, destination, weights, extended_radius)
        recommendations = optimizer.get_recommendations()

        if recommendations.empty:
            print(f"No parking lots are available within {extended_radius} meters radius.")
            map_viz = optimizer.visualize_results(recommendations)
            map_viz.save('parking_recommendation.html')
            return

    current_radius = optimizer.search_radius
    best_lot = recommendations.iloc[0]

    print(f"\nBest Parking Lot Found in {current_radius}m:")
    print(f"ID: {best_lot['id']}")
    print(f"Location: ({best_lot['lat']}, {best_lot['lng']})")
    print(f"Hourly Rate: ${best_lot['rate_hourly']}")
    print(f"Distance to destination: {best_lot['distance']:.0f}m")
    print(f"Fitness Score: {best_lot['fitness']:.3f}")

    if len(recommendations) > 1:
        print("\nOther Available Parking Lots:")
        for _, lot in recommendations.iloc[1:].iterrows():
            print(f"\nID: {lot['id']}")
            print(f"Location: ({lot['lat']}, {lot['lng']})")
            print(f"Rate: ${lot['rate_hourly']}/hr")
            print(f"Distance: {lot['distance']:.0f}m")
            print(f"Fitness Score: {lot['fitness']:.3f}")

    map_viz = optimizer.visualize_results(recommendations)
    map_viz.save('parking_recommendation.html')

## Run the Recommendation

Adjust the `destination` and `search_radius` as desired.

In [5]:
print("Finding parking lot...")
destination =  (43.6532, -79.3832)
parking_recommendation(destination, search_radius=500)

Finding parking lot...

Best Parking Lot Found in 500m:
ID: 16.0
Location: (43.651696, -79.383699)
Hourly Rate: $7.0
Distance to destination: 172m
Fitness Score: 0.551

Other Available Parking Lots:

ID: 15.0
Location: (43.656004, -79.379933)
Rate: $6.0/hr
Distance: 408m
Fitness Score: 0.297

ID: 135.0
Location: (43.65563165773406, -79.3857854604721)
Rate: $8.0/hr
Distance: 341m
Fitness Score: 0.293

ID: 12.0
Location: (43.652705, -79.377218)
Rate: $7.0/hr
Distance: 484m
Fitness Score: 0.182


## Output and Visualization

- The best lot and other recommendations are printed in the cell above.
- A map (`parking_recommendation.html`) is saved in the current directory, which can be opened to visualize the results.
